## DSR Mini-Competition 
### Rossman Sales Prediction
#### Team 1: John Enevoldsen, Sara Ghasemi, Mena Nasr

This is a notebook to reproduce the results that are submitted for the competition. Data understanding and exploration and visualisations are not included in this notebook.

In [1]:
import sys

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import xgboost
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model

In [2]:
sys.path.insert(0, './modules/')
import cleaning as cln
import feature_eng as feng

## 1. Initial Data Preparation

Read the files and make one data frame to be used for training and cross validation, and one to be used for the final test.

In [3]:
#Read the data from 'train', 'store' and 'holdout' files:

full_df_train = pd.read_csv("./data/train.csv")
full_df_store = pd.read_csv("./data/store.csv")
full_df_holdout = pd.read_csv("./data/holdout.csv")

/Users/sara/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#Merge the 'train' and 'store' data frames, to be used for training and cross validation:

full_df_train_cv = cln.merge(full_df_train, full_df_store)

In [5]:
#Merge the 'holdout' and 'store' data frames, to be used for final test:

full_df_test = cln.merge(full_df_holdout, full_df_store)

## 2. Data Cleaning

In [6]:
# Remove the Cusomers column, as was instructed to us for the competition

df_train_cv = cln.drop_column(full_df_train_cv, column='Customers')

df_test = cln.drop_column(full_df_test, column='Customers')

In [7]:
# Target cleaning: Remove the zero and null values in targt ('Sales')

df_train_cv = cln.clean_targets(df_train_cv, target='Sales')

df_test = cln.clean_targets(df_test, target='Sales')

In [8]:
# Feature cleaning:
# Remove the rows with null feature values if number of null values are very small.
# Drop the whole feature column if number of null values are not small.
# The threshold is 10%.
# Transform the values in 'StateHoliday' column: transform 0.0 to 0 and all column to string.
    
df_train_cv = cln.rough_features_cleaning(df_train_cv, threshold=0.10, drop_columns=True, verbose=False)

df_test = cln.rough_features_cleaning(df_test, threshold=0.10, drop_columns=True, verbose=False)

Total number of rows before cleaning:  531983
Total number of rows after cleaning:  425689
Total number of rows before cleaning:  315540
Total number of rows after cleaning:  314760


In [9]:
# Remove the Open column because at this point its values are always equal to 1

df_train_cv = cln.drop_column(df_train_cv, column='Open')

df_test = cln.drop_column(df_test, column='Open')

## 3. Feature Engineering

Some feature engineering (namely mean encoding) are done after split of df_train_cv data set into training and cross validation sets, to avoid data leakage, but others (make new date features, one_hot_encoding, etc.) can be done before. 

The last 3 months of df_train_cv data set (i.e. from 2014-05-01 to 2014-07-31) are used as the cross validation set and the rest of it (i.e. from 2013-01-01 to 2014-04-30) for the training set.

In [10]:
# Make new date features

df_train_cv = feng.dates_features(df_train_cv)

df_test = feng.dates_features(df_test)

In [11]:
# Add one hot encoding of StateHoliday

df_train_cv = feng.one_hot_encoding(df_train_cv, 'StateHoliday')

df_test = feng.one_hot_encoding(df_test, 'StateHoliday')

In [12]:
# Split the df_train_cv data into training and cross validation sets

df_train, df_cv = feng.date_split_train_test(df_train_cv, '2014-05-01')

In [13]:
# Add mean encoding for the Store id on Sales ** only for the train data **
# Then map the same values from train set to the cross validation set and test set

df_train, df_cv = feng.mean_encoding(df_train, df_cv, 'Store')

df_train, df_test = feng.mean_encoding(df_train, df_test, 'Store')

In [14]:
# Checking Store_mean_encoded:

df_train.loc[df_train.loc[:, 'Store'] == 353.0].head(1)

,Date,Store,Sales,Promo,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,month,day_of_week,day_of_month,is_monday,is_saturday,StateHoliday _0,StateHoliday _a,StateHoliday _b,StateHoliday _c,Store_mean_encoded
27,2013-01-01,353.0,3139.0,0.0,1.0,b,b,900.0,1.0,1,1,1,0,0,0,1,0,0,4975.040107


In [15]:
# Checking Store_mean_encoded:

df_cv.loc[df_cv.loc[:, 'Store'] == 353.0].head(1)

,Date,Store,Sales,Promo,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,month,day_of_week,day_of_month,is_monday,is_saturday,StateHoliday _0,StateHoliday _a,StateHoliday _b,StateHoliday _c,Store_mean_encoded
541916,2014-05-02,353.0,5828.0,1.0,0.0,b,b,900.0,1.0,5,4,2,0,0,1,0,0,0,4975.040107


In [16]:
# Checking Store_mean_encoded:

df_test.loc[df_test.loc[:, 'Store'] == 353.0].head(1)

,Date,Store,Sales,Promo,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,month,day_of_week,day_of_month,is_monday,is_saturday,StateHoliday _0,StateHoliday _a,StateHoliday _b,StateHoliday _c,Store_mean_encoded
99,2014-08-01,353,5964,1,1,b,b,900.0,1,8,4,1,0,0,1,0,0,0,4975.040107


In [17]:
# Remove Store columns from data sets as it is mean-encoded now

df_train = cln.drop_column(df_train, column='Store')

df_cv = cln.drop_column(df_cv, column='Store')

df_test = cln.drop_column(df_test, column='Store')

In [18]:
# Add mean encoding for Assortment, StoreType, day_of_week as well, same as explained above

df_train, df_cv = feng.mean_encoding(df_train, df_cv, 'Assortment')
df_train, df_test = feng.mean_encoding(df_train, df_test, 'Assortment')

df_train, df_cv = feng.mean_encoding(df_train, df_cv, 'StoreType')
df_train, df_test = feng.mean_encoding(df_train, df_test, 'StoreType')

df_train, df_cv = feng.mean_encoding(df_train, df_cv, 'day_of_week')
df_train, df_test = feng.mean_encoding(df_train, df_test, 'day_of_week')

df_train = cln.drop_column(df_train, column='StoreType')
df_cv = cln.drop_column(df_cv, column='StoreType')
df_test = cln.drop_column(df_test, column='StoreType')

df_train = cln.drop_column(df_train, column='Assortment')
df_cv = cln.drop_column(df_cv, column='Assortment')
df_test = cln.drop_column(df_test, column='Assortment')

df_train = cln.drop_column(df_train, column='day_of_week')
df_cv = cln.drop_column(df_cv, column='day_of_week')
df_test = cln.drop_column(df_test, column='day_of_week')

## 4. Modeling



Models are evaluated on the **root mean square percentage error (RMSPE)**

In [19]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

In [20]:
# Actual or True value used in the metric in validation process: Sales from cross validation set

actuals_cv = df_cv .Sales.to_numpy()

In [21]:
# Features and targets used for modelling and validatoin:

X_train = df_train.drop(['Date','Sales'], axis=1)
y_train = df_train.loc[:, 'Sales']

X_cv = df_cv.drop(['Date','Sales'], axis=1)
y_cv = df_cv.loc[:, 'Sales']

X_test = df_test.drop(['Date','Sales'], axis=1)
y_test = df_test.loc[:, 'Sales']

### 4.1. Baseline

#### Simple mean

In [22]:
# Baseline Prediction: Mean Sales of the store, taken from train set 
# (don't get confuse by seeing df_cv name! They values are mapped from train set.)

preds_baseline = df_cv.Store_mean_encoded.to_numpy()

print(f'Baseline Model RMSPE = {metric(preds_baseline, actuals_cv):.2f}%')

Baseline Model RMSPE = 31.68%


### 4.2. Multivariate Linear Regression

In [23]:
# Linear Regression model initialisation

model_regr = linear_model.LinearRegression()

In [24]:
# Fit and predict

model_regr.fit(X_train, y_train)
preds_regr = model_regr.predict(X_cv)

In [25]:
# Evaluate

print(f'Linear Regression Model RMSPE = {metric(preds_regr, actuals_cv):.2f}%')

Linear Regression Model RMSPE = 23.61%


### 4.3. Random Forest

In [26]:
# Random forest model initialisation

model_rf = RandomForestRegressor(n_estimators=100, max_depth=5)

In [27]:
# Fit and predict

model_rf.fit(X_train, y_train)
preds_rf = model_rf.predict(X_cv)

In [28]:
# Evaluate

print(f'Random Forest Model RMSPE = {metric(preds_rf, actuals_cv):.2f}%')

Random Forest Model RMSPE = 22.56%


### 4.3. XGBoost tree

In [29]:
# XGBoost model initialisation

model_xgb = xgboost.XGBRegressor(max_depth=12, learning_rate=0.05)

In [30]:
# Fit and predict

model_xgb.fit(X_train, y_train)
preds_xgb = model_xgb.predict(X_cv)

[14:04:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [31]:
# Evaluate

print(f'XGBoost Model RMSPE = {metric(preds_xgb, actuals_cv):.2f}%')

XGBoost Model RMSPE = 15.46%


In [35]:
# Save this as our best model

model_xgb.save_model('./models/xgb_team1.model')

## 5. Testing on the unseen data

The test set (a.k.a. holdout set) gone through the same cleaning and encoding process and our best model (XGBoost) is tested on it.

In [32]:
# Actual or True value used in the metric for final test: Sales from unseen test set

actuals_test = df_test.Sales.to_numpy()

In [33]:
# Prediction from our best model for the unseen test set:

preds_test = model_xgb.predict(X_test)

In [34]:
print(f'Final test RMSPE = {metric(preds_test, actuals_test):.2f}%')

Final test RMSPE = 27.25%
